In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork, Trajectory
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch_geometric.transforms as T

from models import GTNModel, GTCModel, GAEModel, Node2VecModel, GCNEncoder, Traj2VecModel
from evaluation.tasks import TravelTimeEstimation, NextLocationPrediciton

In [2]:
network = RoadNetwork()
network.load("../../osm_data/porto")
trajectory = Trajectory("../../datasets/trajectories/Porto/road_segment_map_final.csv", nrows=10000000).generate_TTE_datatset()

traj_features = pd.read_csv("../../datasets/trajectories/Porto/speed_features_unnormalized.csv")
traj_features.set_index(["u", "v", "key"], inplace=True)
traj_features["util"] = (traj_features["util"] - traj_features["util"].min()) / (traj_features["util"].max() - traj_features["util"].min())  # min max normalization
traj_features["avg_speed"] = (traj_features["avg_speed"] - traj_features["avg_speed"].min()) / (traj_features["avg_speed"].max() - traj_features["avg_speed"].min())  # min max normalization
traj_features.fillna(0, inplace=True)

# data = network.generate_road_segment_pyg_dataset(drop_labels=["highway_enc"])
data_roadclf = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["highway_enc"], traj_data=traj_features.copy())
data_meanspeed = network.generate_road_segment_pyg_dataset(include_coords=True, drop_labels=["avg_speed"], traj_data=traj_features.copy())
data_rest = network.generate_road_segment_pyg_dataset(include_coords=True, traj_data=traj_features.copy())

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1544234 [00:00<?, ?it/s]

In [ ]:
m = Traj2Vec.map_traj_to_node_ids(trajectory["seg_seq"].values, network, 10)

In [ ]:
print(m[-1])

In [10]:
adj = np.loadtxt("./gtn_precalc_adj/traj_adj_k_1.gz") # for traj2vec 'traj_adj_k_1_False_no_selfloops_smoothed'

In [ ]:
print(adj[108, 130:140])

In [ ]:
walks = Traj2Vec.traj_walk(adj, 5, 10000*[0], 10)
print(walks)

In [ ]:
from _walker import random_walks as _random_walks
from scipy import sparse

A = sparse.csr_matrix(adj)
indptr = A.indptr.astype(np.uint32)
indices = A.indices.astype(np.uint32)
weights = A.data.astype(np.float32)

_random_walks(indptr, indices, weights, [100,100,100], 5, 6)

In [ ]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
traj2vec = Traj2VecModel(
            data,
            network,
            adj,
            device=device,
            emb_dim=128,
            walk_length=30,
            context_size=5,
            walks_per_node=25,
            num_neg=10,
        )
traj2vec.train(epochs=200)

In [ ]:
torch.save(traj2vec.state_dict(), "modelt.pt")

In [ ]:
data.x = None
data = T.OneHotDegree(128)(data)

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
# precalc adj matrices
GTCModel(data_rest, device, network, trajectory, k=6, bidirectional=False, add_self_loops=True)

100%|██████████| 1544234/1544234 [14:12<00:00, 1810.84it/s]


In [16]:
models = []
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = GTCModel(data_roadclf, device, network, trajectory, adj=adj)
# model2 = GTNModel(data2, device, network, trajectory, load_traj_adj_path="./gtn_precalc_adj/traj_adj_k_1.gz", norm=True)
# model3 = GAEModel(data2, device=device, encoder=GCNEncoder, emb_dim=128, layers=1)
# model4 = GAEModel(data2, device=device, encoder=GCNEncoder, emb_dim=128, layers=1)
model5 = Node2VecModel(data_roadclf, device=device, q=4, p=1)
model6 = Traj2VecModel(data_roadclf, network, adj, device=device, emb_dim=128, walk_length=30, context_size=5, walks_per_node=25, num_neg=10)

models.extend([(model, 5000)]) # (model3, 5000), (model4, 5000)

In [ ]:
model.train_data.edge_weight.shape

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
models = []
# for k in [1]:
#     model = GTNModel(data, device, network, trajectory, load_traj_adj_path="./traj_adj_k_{}.gz".format(k))
#     model.train(epochs=1000)
#     models.append(model)

model3.train(epochs=50)
model.train(epochs=5000)
model2.train(epochs=5000)
models.extend([model, model2, model3])

In [ ]:
z = model.model(model.train_data.x, model.train_data.edge_traj_index, model.train_data.edge_weight)
z.shape

In [ ]:
print(models)

In [ ]:
model.save_model(path="../model_states/gtn/test")

In [14]:
from torch_geometric.nn.norm import LayerNorm
# load node2vec emb
model5.load_model("../model_states/node2vec/model.pt")
z2 = model5.load_emb()
model6.load_model("../model_states/traj2vec/model.pt")
z3 = model6.load_emb()

norm = LayerNorm(z3.shape[1], affine=False)
z4 = norm(torch.Tensor(z2)).detach().cpu().numpy()
z5 = norm(torch.Tensor(z3)).detach().cpu().numpy()

In [ ]:
print(z3.shape)

In [17]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

n2v = models[-1]
idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)
y = np.array([network.gdf_edges.loc[n]["highway_enc"] for n in network.line_graph.nodes])

for m, e in models:
    m.train(epochs=e)
    zn = m.load_emb()
    zcn = np.concatenate((zn, z2), axis=1)
    zct = np.concatenate((zn, z3), axis=1)
    # zcnn = np.concatenate((zn, z4), axis=1)
    # zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
    eva = [zn, zcn, zct]
    for X in eva:
        X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

        lm = linear_model.LogisticRegression(multi_class="multinomial", max_iter=1000)
        lm.fit(X_train, y_train)
        scorer = make_scorer(metrics.f1_score, average="macro")
        print(np.mean(cross_val_score(estimator=lm, X=X_test, y=y_test, scoring=scorer, cv=5)))
    #print(metrics.classification_report(y_test, lm.predict(X_test)))

/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.3344667406978582


/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.47672869835943193


/home/pheinemeyer/miniconda3/envs/road/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


0.4908523362647498


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

tf = pd.read_csv("../../datasets/trajectories/Porto/speed_features_unnormalized.csv")
tf.set_index(["u", "v", "key"], inplace=True)
map_id = {j: i for i, j in enumerate(network.line_graph.nodes)}
tf["idx"] = tf.index.map(map_id)
tf.sort_values(by="idx", axis=0, inplace=True)

idxs = np.arange(len(network.line_graph.nodes))
train_idx, test_idx = model_selection.train_test_split(idxs, test_size=0.2, random_state=69)

y = tf["avg_speed"]
y.fillna(0, inplace=True)
y = y.round(2)
y = y.values

for m, e in models:
    m.train(epochs=e)
    
    zn = m.load_emb()
    zcn = np.concatenate((zn, z2), axis=1)
    zct = np.concatenate((zn, z3), axis=1)
    zcnn = np.concatenate((zn, z4), axis=1)
    zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
    eva = [zn, zcn, zct, zcnn, zctn]
    for X in eva:
        decoder = linear_model.LinearRegression(fit_intercept=True)

        X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]

        decoder.fit(X_train, y_train)
        scorer = make_scorer(metrics.mean_absolute_error)
        print(np.mean(cross_val_score(estimator=decoder, X=X_test, y=y_test, scoring=scorer, cv=5)))

In [ ]:
travel_time_est = TravelTimeEstimation(
    traj_dataset=trajectory,
    network=network,
    device=device,
    batch_size=128,
    epochs=5,
    seed=88,
)
travel_time_est.register_metric(
    name="MSE", metric_func=metrics.mean_squared_error, args={}
)
travel_time_est.register_metric(
    name="MAE", metric_func=metrics.mean_absolute_error, args={}
)

for i, (m, e) in enumerate(models):
    m.train(epochs=e)
    zn = m.load_emb()
    zcn = np.concatenate((zn, z2), axis=1)
    zct = np.concatenate((zn, z3), axis=1)
    # X = z # embedding for each node
    eva = [zn, zcn, zct]
    for X in eva:
        print(travel_time_est.evaluate(X))

In [ ]:
nextlocation_pred = NextLocationPrediciton(
    traj_dataset=trajectory,
    network=network,
    device=device,
    batch_size=256,
    epochs=5,
    seed=88,
)

nextlocation_pred.register_metric(
    name="accuracy",
    metric_func=metrics.accuracy_score,
    args={"normalize": True},
)

for i, (m, e) in enumerate(models):
    m.train(epochs=e)
    zn = m.load_emb()
    zcn = np.concatenate((zn, z2), axis=1)
    zct = np.concatenate((zn, z3), axis=1)
    zcnn = np.concatenate((zn, z4), axis=1)
    zctn = np.concatenate((zn, z5), axis=1)
    # X = z # embedding for each node
    eva = [zn, zcn, zct, zcnn, zctn]
    for X in eva:
        print(nextlocation_pred.evaluate(X))